# Blending ensembling

In [1]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))

Cargo el df con los features.

In [3]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [4]:
assert(df_features.shape[0] == get_clean_df()['person'].unique().shape[0])

Cargo el set de entrenamiento.

In [5]:
%run $set_de_entrenamiento_testing_y_prediccion

labels_with_features = labels.merge(df_features, how='inner', on='person')
train = labels_with_features.drop('label', axis=1)
train_target = labels_with_features['label']

### Predictores base

En esta sección vamos a preparar los predictores base a utilizar. Estos son los mismos que se encuentran en la carpeta *Algoritmos de ML*.

In [6]:
base_predictors = []

### AdaBoost


In [7]:
param = {
    'n_estimators': 128,
    'loss': 'linear',
    'learning_rate': 0.07,
    'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
}

base_predictors.append(AdaBoostRegressor(**param))

### Random Forest

In [8]:
param = {
    'bootstrap': True,
    'max_depth': 9,
    'max_features': 37,
    'min_samples_leaf': 30,
    'min_samples_split': 7,
    'n_estimators': 784
}

base_predictors.append(RandomForestRegressor(**param))

### DecisionTree

In [9]:
base_predictors.append(DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'))

Separo un 10% del set de entrenamiento para luego entrenar el blending.

In [10]:
train, train_blending, train_target, train_blending_target = train_test_split(train, train_target, test_size=0.1, stratify=train_target)

In [12]:
assert(train.shape[0] == train_target.shape[0])
assert(train_blending.shape[0] == train_blending_target.shape[0])

Entreno los predictores base con el 90% del set de entrenamiento (separando este 90% en sets de entrenamiento y validacion para optimizar cada modelo) y realizo las predicciones para el 10% que separamos.

**Nota**: falta hacer el cross validation con el 90% para encontrar los parámetros óptimos. Actualmento los parámetros están optimizados para el 100%. Esto implica una filtración.

## Predicción para el Blending

Ahora vamos a realizar una predicción por cada predictor base, y las vamos a agregar a una copia del set de entrenamiento (*train_blending_predictions*).

In [14]:
train_blending_predictions = train_blending.copy()
for predictor in base_predictors:
    train_blending_predictions[predictor.__class__.__name__] = np.nan

In [15]:
%%time
for predictor in base_predictors:
    # como warm_start=False cada vez que llamo fit, el modelo se reinicia
    predictor.fit(train, train_target) # train
    train_blending_predictions[predictor.__class__.__name__] = predictor.predict(train_blending) # predict

CPU times: user 46 s, sys: 0 ns, total: 46 s
Wall time: 46 s


In [16]:
train_blending_predictions.head(3)

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,Saturday,Sunday,Thursday,Tuesday,Wednesday,tarde,maniana,madrugada,noche,AdaBoostRegressor,RandomForestRegressor,DecisionTreeRegressor
person,,,,,,,,,,,,,,,,,,,,,,,,,
62ae03a2,900.0,1440.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.386309,0.211067,0.205758
16ab6c38,768.0,1366.0,0.0,0.0,2.0,19.0,2.0,0.0,7.0,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.129783,0.015669,0.007792
e03f0729,640.0,360.0,0.0,0.0,8.0,9.0,7.0,0.0,6.0,0.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.144921,0.023193,0.056570


### Modelo Blender

Entrenamos un modelo blender que use las predicciones aprendidas para realizar las nuevas predicciones.

In [18]:
blend_train = train_blending_predictions[[predictor.__class__.__name__ for predictor in base_predictors]]
blend_train.head(3)

,AdaBoostRegressor,RandomForestRegressor,DecisionTreeRegressor
person,,,
62ae03a2,0.386309,0.211067,0.205758
16ab6c38,0.129783,0.015669,0.007792
e03f0729,0.144921,0.023193,0.056570


In [19]:
param = {
    'solver': 'lbfgs'
}
blender = LogisticRegression(**param)

In [21]:
%%time
scores = cross_val_score(blender, blend_train, train_blending_target, cv=10, scoring='roc_auc')
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.846799 (+/- 0.150817)
CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 101 ms
